We import what will serve us

In [1]:
from pandas import DataFrame, read_csv, to_numeric
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import pickle
from bigml.api import BigML
import kaggle

%matplotlib inline

## Loading csv files as data frames

Files must be placed in the same directory as this file. Alternatively, modify the relative path to those files.

In [2]:
!pwd
!ls
fulltrain=read_csv('./cs-training.csv', index_col=0)
test=read_csv('./cs-test.csv', index_col=0)

data_sets=[fulltrain,test] #pour travailler sur les 2 datasets 

/home/devel/handson-ml/projet_appliquer/give_me_some_credit
100_biggest_errors.csv
100_most_big_errors.csv
3_evaluation_bigml_give_me_credit.ipynb
Credit-Model-Create-sklearn.ipynb
Credit-Split.ipynb
README.md
big_ml_id.txt
cs-test.csv
cs-training.csv
dataset_deepnet_ensemble
docker
ensemble_deepnet_comparison-Copy1.ipynb
ensemble_deepnet_comparison.ipynb
ensemble_deepnet_comparison_evaluation.ipynb
ensemble_deepnet_comparison_visualisation.ipynb
eval_christophe.joblib
eval_laurent.joblib
eval_maud.joblib
eval_maxime.joblib
evaluation_bigml_give_me_credit2.ipynb
evaluation_dataset3.csv
evaluation_dataset3_with_proba.csv
field_Importances
kaggle
kaggleprediction_3.csv
le_bordel.ipynb
methodo_kaggle.txt
new_features_and_bigml_give_me_credit.ipynb
new_features_give_me_credit.ipynb
predictions
recap_kaggle.ipynb
retour_eval.py
sauvegarde.txt
testset
training_bigml_give_me_credit.ipynb
trainingset
validation3.csv
validation_and_kaggle_give_me_credit.ipynb


In [3]:
project = 'gmsc'
version = 'v1.2'

In [6]:
comp_csv_suffix = version + '.csv.bz2'
csv_suffix = version + '.csv'

## Correcting the values in monthly income
Set NaN to 0 in the monthly income column

In [7]:
for df in data_sets:
    df.loc[:,'MonthlyIncome'].fillna(0, inplace=True)
    
data_sets[0].head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0


## Creating a new column for monthly debt.

In [8]:
for df in data_sets:
    df.insert(6,"MonthlyDebt",0)
data_sets[0].head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,0,9,0,4,0,2.0


We calculate the monthly debt.
If the income is zero, we take the monthly debt from the debt ratio.

In [9]:
for df in data_sets:
    df.loc[:,'MonthlyDebt']=df['DebtRatio']*df['MonthlyIncome']
    df.loc[df['MonthlyIncome'] == 0,'MonthlyDebt']=df.loc[df['MonthlyIncome'] == 0,'DebtRatio']
   
    df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=None #0.8610
   
data_sets[0][['MonthlyDebt','DebtRatio','MonthlyIncome']].head(10)

,MonthlyDebt,DebtRatio,MonthlyIncome
1,7323.197016,0.802982,9120.0
2,316.878123,0.121876,2600.0
3,258.914887,0.085113,3042.0
4,118.963951,0.036050,3300.0
5,1584.975094,0.024926,63588.0
6,1314.624392,0.375607,3500.0
7,5710.000000,NaN,0.0
8,734.790059,0.209940,3500.0
9,46.000000,NaN,0.0
10,14359.393699,0.606291,23684.0


# Create New column disposable income

In [10]:
for df in [fulltrain, test]:
    df.insert(7,"DisposableIncome",0)
fulltrain.head(10)


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,0,3,0,1,0,1.0
7,0,0.305682,57,0,NaN,0.0,5710.000000,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,0,8,0,0,0,0.0
9,0,0.116951,27,0,NaN,0.0,46.000000,0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,0,9,0,4,0,2.0


In [11]:
for df in data_sets:
    df.loc[:,'DisposableIncome']=df['MonthlyIncome']-df['MonthlyDebt']
    df.loc[df['MonthlyIncome'] == 0,'DisposableIncome']=0
    
data_sets[0][['MonthlyDebt','DebtRatio','MonthlyIncome','DisposableIncome']].head(10)

,MonthlyDebt,DebtRatio,MonthlyIncome,DisposableIncome
1,7323.197016,0.802982,9120.0,1796.802984
2,316.878123,0.121876,2600.0,2283.121877
3,258.914887,0.085113,3042.0,2783.085113
4,118.963951,0.036050,3300.0,3181.036049
5,1584.975094,0.024926,63588.0,62003.024906
6,1314.624392,0.375607,3500.0,2185.375608
7,5710.000000,NaN,0.0,0.000000
8,734.790059,0.209940,3500.0,2765.209940
9,46.000000,NaN,0.0,0.000000
10,14359.393699,0.606291,23684.0,9324.606301


## Correcting the values in number of dependents
Set to 0 the number of dependents when not available.

In [12]:
for df in data_sets:
    df.loc[:,'NumberOfDependents'].fillna(0, inplace=True)
    df.loc[:,'NumberOfDependents']=to_numeric(df['NumberOfDependents'],downcast='integer')
    
data_sets[0][['NumberOfDependents']].head(10)

,NumberOfDependents
1,2
2,1
3,0
4,0
5,0
6,1
7,0
8,0
9,0
10,2


# Disposable Income / dept +1


Create new colomn BalancedIncomePerDependent

In [13]:
 for df in data_sets:
    df.insert(8,"BalancedIncomePerDependent",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,0,13,0,6,0,2
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,0,4,0,0,0,1
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,0,2,1,0,0,0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,0,5,0,0,0,0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,0,7,0,1,0,0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,0,3,0,1,0,1
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,0,8,0,3,0,0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,0,8,0,0,0,0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,0,2,0,0,0,0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,0,9,0,4,0,2


In [14]:
for df in data_sets:
    df.loc[:,'BalancedIncomePerDependent']= df['DisposableIncome'] / (df['NumberOfDependents']+1)
      
fulltrain['BalancedIncomePerDependent'].head(10)

1       598.934328
2      1141.560939
3      2783.085113
4      3181.036049
5     62003.024906
6      1092.687804
7         0.000000
8      2765.209940
9         0.000000
10     3108.202100
Name: BalancedIncomePerDependent, dtype: float64

# Create new colomn BalancedIncomePerDependent 

Create new colomn  BalancedIncome  MontlyIncome/ (NumberOfDependents +1)

In [15]:
for df in [fulltrain, test]:
    df.insert(8,"BalancedIncome",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,0,598.934328,13,0,6,0,2
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,0,1141.560939,4,0,0,0,1
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,0,2783.085113,2,1,0,0,0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,0,3181.036049,5,0,0,0,0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,0,62003.024906,7,0,1,0,0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,0,1092.687804,3,0,1,0,1
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,0,0.000000,8,0,3,0,0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,0,2765.209940,8,0,0,0,0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,0,0.000000,2,0,0,0,0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,0,3108.202100,9,0,4,0,2


In [16]:
for df in data_sets:
    df.loc[:,'BalancedIncome']= df['MonthlyIncome'] / (df['NumberOfDependents']+1)
 
data_sets[0]['BalancedIncome'].head(10)

1      3040.000000
2      1300.000000
3      3042.000000
4      3300.000000
5     63588.000000
6      1750.000000
7         0.000000
8      3500.000000
9         0.000000
10     7894.666667
Name: BalancedIncome, dtype: float64

# Weighted sum of late payments

In [17]:
for df in data_sets:
    df.insert(13,"WeightedOfLatePayment",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,WeightedOfLatePayment,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,3040.000000,598.934328,13,0,6,0,0,2
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,1300.000000,1141.560939,4,0,0,0,0,1
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,3042.000000,2783.085113,2,1,0,0,0,0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,3300.000000,3181.036049,5,0,0,0,0,0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,63588.000000,62003.024906,7,0,1,0,0,0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,1750.000000,1092.687804,3,0,1,0,0,1
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,0.000000,0.000000,8,0,3,0,0,0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,3500.000000,2765.209940,8,0,0,0,0,0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,0.000000,0.000000,2,0,0,0,0,0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,7894.666667,3108.202100,9,0,4,0,0,2


In [18]:
for df in data_sets:
    df['WeightedOfLatePayment']=   (df['NumberOfTime30-59DaysPastDueNotWorse']*1) + (df['NumberOfTime60-89DaysPastDueNotWorse']*2) + (df['NumberOfTimes90DaysLate']*3)

In [19]:
data_sets[0]

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,WeightedOfLatePayment,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,3040.000000,598.934328,13,0,6,2,0,2
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,1300.000000,1141.560939,4,0,0,0,0,1
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,3042.000000,2783.085113,2,1,0,4,0,0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,3300.000000,3181.036049,5,0,0,0,0,0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,63588.000000,62003.024906,7,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,0,0.040674,74,0,0.225131,2100.0,472.774869,1627.225131,2100.000000,1627.225131,4,0,1,0,0,0
149997,0,0.299745,44,0,0.716562,5584.0,4001.283436,1582.716564,1861.333333,527.572188,4,0,1,0,0,2
149998,0,0.246044,58,0,NaN,0.0,3870.000000,0.000000,0.000000,0.000000,18,0,1,0,0,0
149999,0,0.000000,30,0,0.000000,5716.0,0.000000,5716.000000,5716.000000,5716.000000,4,0,0,0,0,0


# Replace the outlier of DebtRatio by 0.3

In [29]:
for df in data_sets:
    df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=0.3

data_sets[0]['DebtRatio'].head(10)

1     0.802982
2     0.121876
3     0.085113
4     0.036050
5     0.024926
6     0.375607
7     0.300000
8     0.209940
9     0.300000
10    0.606291
Name: DebtRatio, dtype: float64

# Rearrange columns

We test 6 different configurations

fulltrain1 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test1 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]


fulltrain2 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test2 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]



In [34]:
fulltrain = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]


In [35]:
filename_fulltrain = 'gmsc_fulltrain_' + comp_csv_suffix
filename_test = 'gmsc_test-' + comp_csv_suffix


In [36]:
export_csv = fulltrain.to_csv (r'lmcm/'+ filename_fulltrain,header=True,index_label='Id')
export_csv = test.to_csv (r'lmcm/'+ filename_test,header=True,index_label='Id')

fulltrain4 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test4 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]


fulltrain5 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test5 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines',  'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]


fulltrain6 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test6 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate',  'NumberOfDependents', 'SeriousDlqin2yrs']]


#### We use RandomState for have a fixed seed and always split the training in the same way

In [38]:
from sklearn.model_selection import train_test_split

filename_train = 'gmsc_train80-' + comp_csv_suffix
filename_dev = 'gmsc_dev20-' + comp_csv_suffix
fixed_state = 54321

training_set, dev_set = train_test_split(fulltrain, test_size=0.2, random_state=fixed_state)

export_csv = training_set.to_csv (r'lmcm/'+ filename_train,header=True,index_label='Id') 
export_csv = dev_set.to_csv (r'lmcm/'+ filename_dev,header=True,index_label='Id') 

tset_shape = trainingSet3.shape
nbr_ligne = int(tset_shape[0]/20)
nbr_ligne

nbr_ligne = nbr_ligne + 1
start=0
end=int(tset_shape[0]/20)
n_dataset = 1

nbr_ligne_total
while start <= tset_shape[0]:
    trainingSet3[start:end].to_csv (r'dataset_deepnet_ensemble/training_set' + str(n_dataset) + '.csv',  header=True)
    n_dataset = n_dataset+1
    start = start + nbr_ligne
    end = end + nbr_ligne


Maud SONET